In [1]:
from pylab import *
import matplotlib.pyplot as plt
import random
import numpy as np
from queryParser import *
import psycopg2
import time

# input="SELECT SUM(l.lo_extendedprice * l.lo_discount) AS revenue FROM lineorder l , date d WHERE l.lo_orderdate = d.d_datekey AND d.d_year = 1993 AND l.lo_discount BETWEEN 1 AND 3 AND l.lo_quantity < 25;"
dimentions={"date":0,"supplier":0,"customer":0,"part":0}
tablesWithSel={}
sortedTables=[]
FACT="lineorder"
def MinSelectivity(input,cursor):
    
    state=[]
    solution={}
    joinedTables ,parsed_query , alias=queryParser(input)
    joinedTables.remove(FACT)
    tables=get_tableWithSelectivity(parsed_query)
    
    #calculer cardinalités des dimentions
    for t in dimentions:
        query="select count(*) from "+t
        print("query",query)
        cursor.execute(query)
        dimentions[t]=cursor.fetchall()[0][0]
    
    # calculer selectivités des dimentions
    for key in tables.keys():
        json={'select': {'value': {'count': '*'}},'from':{'value':alias[key],'name':key}}
        if len(tables[key])==1:
            json['where']=tables[key][0]
        else:
            json['where']={'and':tables[key]}
        query=format(json)
        cursor.execute(query)
        tablesWithSel[alias[key]]=cursor.fetchall()[0][0]
        
    # ajouter les tables sans selectivité
    if(len(joinedTables)>len(tablesWithSel)):
        for t in joinedTables:
            if t not in tablesWithSel:
                tablesWithSel[t]=dimentions[t]
  
    sortedTables={k: v for k, v in sorted(tablesWithSel.items(), key=lambda item: item[1])}
    state=list(sortedTables.keys())
    state.insert(1,FACT)
    print(state)
    
    solution["query"]=listToQuery(state,get_indice(state),parsed_query)
    #solution["runTime"]=get_runTime(solution["query"],cursor)
    solution["estimatedCost"]=get_cost(solution["query"],cursor)
    #time.sleep(5)
    #solution["energy"]=get_energy(solution["query"],cursor)
    return solution



In [2]:
conn,cursor = connect_bdd("ssb")
input="SELECT d.d_year,s.s_city,p.p_brand1,SUM(l.lo_revenue - l.lo_supplycost) AS profit  FROM date d,customer c,supplier s,part p,lineorder l WHERE l.lo_custkey = c.c_custkey AND l.lo_suppkey = s.s_suppkey  AND l.lo_partkey = p.p_partkey AND l.lo_orderdate = d.d_datekey AND  c.c_region = 'AMERICA' AND s.s_nation = 'UNITED STATES' AND (d.d_year = 1997 OR d.d_year = 1998)  AND p.p_category = 'MFGR#14' GROUP BY  d.d_year, s.s_city, p.p_brand1 ORDER BY d.d_year,s.s_city,p.p_brand1;"
#input="SELECT c_city,s_city,d_year,SUM(lo_revenue) AS revenue FROM customer c,lineorder l,supplier s,date d WHERE  l.lo_custkey = c.c_custkey AND l.lo_suppkey = s.s_suppkey AND l.lo_orderdate = d.d_datekey AND (c.c_city='UNITED KI1' OR c.c_city='UNITED KI5') AND (s.s_city='UNITED KI1' OR s.s_city='UNITED KI5') AND d.d_yearmonth = 'Dec1997'  GROUP BY  c.c_city, s.s_city, d.d_year ORDER BY  d.d_year ASC,  revenue DESC;"
solution = MinSelectivity(input,cursor)
print(solution)


query select count(*) from date
query select count(*) from supplier
query select count(*) from customer
query select count(*) from part
['date', 'lineorder', 'supplier', 'part', 'customer']
{'query': "SELECT d.d_year, s.s_city, p.p_brand1, SUM(l.lo_revenue - l.lo_supplycost) AS profit FROM date AS d JOIN lineorder AS l ON l.lo_orderdate = d.d_datekey JOIN supplier AS s ON l.lo_suppkey = s.s_suppkey JOIN part AS p ON l.lo_partkey = p.p_partkey JOIN customer AS c ON l.lo_custkey = c.c_custkey WHERE c.c_region = 'AMERICA' AND s.s_nation = 'UNITED STATES' AND (d.d_year = 1997 OR d.d_year = 1998) AND p.p_category = 'MFGR#14' GROUP BY d.d_year, s.s_city, p.p_brand1 ORDER BY d.d_year,s.s_city,p.p_brand1", 'estimatedCost': 3698548.23}


In [ ]:
SELECT   c_nation,s_nation,d_year,SUM(lo_revenue) AS revenue FROM 
  customer c,
  lineorder l,
  supplier s,
  date d
WHERE 
  l.lo_custkey = c.c_custkey
  AND l.lo_suppkey = s.s_suppkey
  AND l.lo_orderdate = d.d_datekey
  AND c.c_region = 'ASIA'
  AND s.s_region = 'ASIA'
  AND d.d_year >= 1992
  AND d.d_year <= 1997
GROUP BY 
  c_nation,
  s_nation,
  d_year
ORDER BY 
  d_year ASC, 
  revenue DESC;